In [1]:
using Pipe

In [2]:
addprocs(["heathred"])
addprocs(["amon"], dir="")

expr: syntax error
expr: syntax error
expr: syntax error


1-element Array{Any,1}:
 8

In [3]:
data = [float([eye(100),eye(100)])[:] for _ in 1:9000]

(sizeof(data[1])*length(data))/(1000*1000)

1440.0

get_chunks (generic function with 1 method)

In [11]:
function get_chunks(data::Vector, nchunks::Int)
    base_len, remainder = divrem(length(data),nchunks)
    chunk_len = fill(base_len,nchunks)
    chunk_len[1:remainder]+=1 #remained will always be less than nw
    function _it() 
        for ii in 1:nchunks
            chunk_start = sum(chunk_len[1:ii-1])+1
            chunk_end = chunk_start + chunk_len[ii] -1
            chunk = data[chunk_start: chunk_end]
            produce(chunk)
        end
    end
    Task(_it)
end

function chunk_data(data_name::Symbol, data::Vector)
    function save_chunk(chunk::Vector)
        eval(:(global $data_name; $data_name=$chunk));
        nothing
    end

    chunks = get_chunks(data, nworkers())
    @sync for (pid,chunk) in zip(workers(),chunks)
        remotecall(pid, save_chunk, chunk) 
    end
end

function prechunked_mapreduce(data_name::Symbol, map_fun::Function, red_acc::Function)
    function do_mapred(dummy)
        @pipe data_name |> eval |> map(map_fun,_) |> reduce(red_acc, _)
        #reduce(red_acc,map(map_fun,eval(data_name)))
    end
    
    rem_results = @pipe workers() |> map(pid->remotecall(pid,do_mapred, nothing), _)
    @pipe rem_results |> map(fetch,_) |> reduce(red_acc, _)
end

chunk_data (generic function with 1 method)

prechunked_mapreduce (generic function with 1 method)

In [13]:
chunk_data(:data, data)
@everywhere println(length(data))

9000
	From worker 2:	1286
	From worker 7:	1285
	From worker 6:	1286
	From worker 8:	1285
	From worker 3:	1286
	From worker 4:	1286
	From worker 5:	1286


In [16]:
@time prechunked_mapreduce(:data, mean, (+))

elapsed time: 0.051323719 seconds (45060 bytes allocated)


90.00000000000007

In [17]:
@time reduce(+,map(mean,data))

elapsed time: 0.177577029 seconds (216144 bytes allocated)


90.00000000000007

In [ ]:
function test()
    @parallel (+) for ii in data
        mean(ii)
    end
end

In [ ]:
Profile.clear
Profile.init(delay=0.01)

@profile test()


In [ ]:
@time test()

In [ ]:
using ProfileView
ProfileView.view()

In [ ]:
function test_one()
    total = 0
    for ii in data
        total+=mean(ii)
    end
end

In [ ]:

@time test_one()

In [ ]:
ProfileView.view()